<a href="https://colab.research.google.com/github/Requenamar3/Data-Mining/blob/main/Data%20Mining%20Final%20Project%20Eccom%20MR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##EDA

In [73]:
import pandas as pd
import matplotlib.pyplot as plt
import mlxtend.frequent_patterns
import mlxtend.preprocessing
import numpy as np
import datetime as dt
!pip install matplotlib
!pip install mlxtend -qqq
!pip install ydata_profiling


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**check out the columns in the dataset in order to determine what columns to load**

In [132]:
base_url = 'https://raw.githubusercontent.com/Requenamar3/Data-Mining/main/'

# List of CSV file paths on GitHub
file_paths = [
    'sales_2023-01-01_2023-04-30.csv',
    'sales_2023-05-01_2023-09-30.csv',
    'sales_2024-03-07_2024-04-01.csv',
    'sales_2023-10-01_2024-03-06.csv','sales_2024-03-07_2024-04-01.csv',
]

# Function to get columns from a single CSV file
def get_columns_from_csv(file_path):
    full_url = f'{base_url}{file_path}'
    # Read just the first row to get the columns
    df = pd.read_csv(full_url, nrows=1)
    return df.columns.tolist()

# Check columns in each dataset
all_columns = set()
for file_path in file_paths:
    columns = get_columns_from_csv(file_path)
    all_columns.update(columns)

# print all_columns and decide which ones to load
print(all_columns)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'month', 'financial_status', 'order_name', 'shipping_postal_code', 'day', 'variant_sku', 'customer_type', 'customer_id', 'customer_cohort_week', 'shipping_region', 'customer_cohort_month', 'units_per_transaction', 'product_price'}


**load only the columns needed**

In [143]:
import pandas as pd

base_url = 'https://raw.githubusercontent.com/Requenamar3/Data-Mining/main/'
file_paths = [
    'sales_2023-01-01_2023-04-30.csv',
    'sales_2023-05-01_2023-09-30.csv',
    'sales_2024-03-07_2024-04-01.csv',
    'sales_2023-10-01_2024-03-06.csv','sales_2024-03-07_2024-04-01.csv',
]

columns_to_load = [
     'customer_type','customer_cohort_month', 'customer_cohort_week',
    'order_name', 'month', 'units_per_transaction','customer_id',
    'product_price', 'shipping_region','variant_sku','financial_status'
]

dataframes_list = []

for file_path in file_paths:
    full_url = f'{base_url}{file_path}'
    # First, check which columns exist in the CSV
    available_columns = pd.read_csv(full_url, nrows=0).columns.tolist()
    # Determine the intersection of available columns and columns to load
    columns_to_read = [column for column in columns_to_load if column in available_columns]
    df = pd.read_csv(full_url, usecols=columns_to_read)
    dataframes_list.append(df)

BBORDERS = pd.concat(dataframes_list, ignore_index=True)
BBORDERS.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_id,customer_type,variant_sku,product_price,shipping_region,customer_cohort_week,customer_cohort_month,units_per_transaction
0,2023-01,partially_refunded,#658025,5.210000e+12,Returning,SQ1726938,287.93,Texas,2021-W27,2021-07,1
1,2023-01,refunded,#667324,5.920000e+12,Returning,NaN,0.00,Florida,2022-W30,2022-07,0
2,2023-01,refunded,#667324,5.920000e+12,Returning,SQ9178530,269.94,Florida,2022-W30,2022-07,1
3,2023-01,refunded,#655100,5.450000e+12,Returning,SQ8862610,559.99,Indiana,2021-W49,2021-12,0
4,2023-01,refunded,#655100,5.450000e+12,Returning,NaN,0.00,Indiana,2021-W49,2021-12,0


**Need to check how recurrent customers (not suscribiers) buying one time order show in the report.**

In [144]:
BB_filtered = BB_filtered.dropna(subset=['variant_sku'])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [135]:
#Need to convert the state names to two-letter code in order to simplify data, save space and ensure consistency
#adding a column for state code
# Dictionary for state name to state code mapping
state_codes = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

BB_filtered['state_code'] = BB_filtered['shipping_region'].map(state_codes)

BB_filtered


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_id,customer_type,variant_sku,product_price,shipping_region,customer_cohort_week,customer_cohort_month,units_per_transaction,state_code
362,2023-01,paid,#658392,6.420000e+12,First-time,SQ8862610,54.99,New York,2023-W01,2023-01,1,NY
377,2023-01,paid,#670349,6.450000e+12,First-time,BBNYBGLA,69.99,Virginia,2023-W04,2023-01,1,VA
393,2023-01,paid,#666714,6.440000e+12,First-time,BB648B,109.99,Pennsylvania,2023-W03,2023-01,1,PA
430,2023-01,paid,#661459,6.430000e+12,First-time,BB062-25,49.99,Connecticut,2023-W02,2023-01,1,CT
459,2023-01,paid,#661431,6.430000e+12,First-time,BB736-37,64.99,Texas,2023-W02,2023-01,1,TX
...,...,...,...,...,...,...,...,...,...,...,...,...
497163,NaN,paid,#891911,7.350000e+12,First-time,BB719-50,99.99,Wisconsin,2024-W14,2024-04,1,WI
497164,NaN,paid,#892251,7.360000e+12,First-time,BB660B,84.99,Arizona,2024-W14,2024-04,1,AZ
497167,NaN,paid,#892262,7.360000e+12,First-time,BB1151-29,69.99,Wisconsin,2024-W14,2024-04,1,WI
497193,NaN,paid,#892276,7.180000e+12,First-time,SQ9178530,179.97,Nevada,2024-W14,2024-04,1,NV


In [136]:
# Filter out rows with a 'product_price' of 0.0 from 'BB_filtered'
BB_filtered = BB_filtered.loc[BB_filtered['product_price'] != 0.0]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [137]:
BB_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23878 entries, 362 to 497196
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   month                  21830 non-null  object 
 1   financial_status       23878 non-null  object 
 2   order_name             23878 non-null  object 
 3   customer_id            23878 non-null  float64
 4   customer_type          23878 non-null  object 
 5   variant_sku            23878 non-null  object 
 6   product_price          23878 non-null  float64
 7   shipping_region        23878 non-null  object 
 8   customer_cohort_week   23878 non-null  object 
 9   customer_cohort_month  23878 non-null  object 
 10  units_per_transaction  23878 non-null  int64  
 11  state_code             23713 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 2.4+ MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [138]:
#Import the ProfileReport .For creating comprehensive exploratory data analysis reports.
from ydata_profiling import ProfileReport

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [139]:
# Create a ProfileReport object
profile = ProfileReport(BB_filtered, title="Ecommerce", explorative=True)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [113]:
# display report
profile

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [86]:
#checking for missing values
BB_filtered.isnull().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


month                    2048
financial_status            0
order_name                  0
customer_type               0
variant_sku                 0
product_price               0
shipping_region             0
customer_cohort_week        0
customer_cohort_month       0
units_per_transaction       0
state_code                165
dtype: int64

Trying to preserve the majority of the records I inspected the data looking for ways of filling missing values with the correct information.

In [114]:
# Filter the DataFrame 'BB_filtered' to find rows where 'month' is NaN (missing)
missing_month_records = BB_filtered[BB_filtered['month'].isnull()]

# Display the records with missing 'month' values
missing_month_records


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_id,customer_type,variant_sku,product_price,shipping_region,customer_cohort_week,customer_cohort_month,units_per_transaction,state_code
291951,NaN,paid,#880945,7.300000e+12,First-time,BB1249-24,69.99,California,2024-W10,2024-03,1,CA
291952,NaN,paid,#880919,7.300000e+12,First-time,BB303P,99.99,Washington,2024-W10,2024-03,1,WA
292023,NaN,paid,#880930,7.300000e+12,First-time,BB343-26,59.99,Georgia,2024-W10,2024-03,1,GA
292049,NaN,paid,#880924,7.300000e+12,First-time,SQ5173256,188.97,Texas,2024-W10,2024-03,1,TX
292198,NaN,paid,#880894,7.300000e+12,First-time,BB1291P,99.99,Virginia,2024-W10,2024-03,1,VA
...,...,...,...,...,...,...,...,...,...,...,...,...
497163,NaN,paid,#891911,7.350000e+12,First-time,BB719-50,99.99,Wisconsin,2024-W14,2024-04,1,WI
497164,NaN,paid,#892251,7.360000e+12,First-time,BB660B,84.99,Arizona,2024-W14,2024-04,1,AZ
497167,NaN,paid,#892262,7.360000e+12,First-time,BB1151-29,69.99,Wisconsin,2024-W14,2024-04,1,WI
497193,NaN,paid,#892276,7.180000e+12,First-time,SQ9178530,179.97,Nevada,2024-W14,2024-04,1,NV


In [115]:
# Grouping 'missing_month_records' by 'customer_cohort_month' and counting records in each group

missing_month_records.groupby('customer_cohort_month').size()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


customer_cohort_month
2024-03    1954
2024-04      94
dtype: int64

**After checking the orders in ShipStation, I can confirm the Month and Customer_cohort_month is the same for those orders.**

In [118]:
# Replace NaN values in 'month' column with values from 'customer_cohort_month'
BB_filtered['month'] = BB_filtered.apply(
    lambda row: row['customer_cohort_month'] if pd.isnull(row['month']) else row['month'],
    axis=1
)

BB_filtered

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_id,customer_type,variant_sku,product_price,shipping_region,customer_cohort_week,customer_cohort_month,units_per_transaction,state_code
362,2023-01,paid,#658392,6.420000e+12,First-time,SQ8862610,54.99,New York,2023-W01,2023-01,1,NY
377,2023-01,paid,#670349,6.450000e+12,First-time,BBNYBGLA,69.99,Virginia,2023-W04,2023-01,1,VA
393,2023-01,paid,#666714,6.440000e+12,First-time,BB648B,109.99,Pennsylvania,2023-W03,2023-01,1,PA
430,2023-01,paid,#661459,6.430000e+12,First-time,BB062-25,49.99,Connecticut,2023-W02,2023-01,1,CT
459,2023-01,paid,#661431,6.430000e+12,First-time,BB736-37,64.99,Texas,2023-W02,2023-01,1,TX
...,...,...,...,...,...,...,...,...,...,...,...,...
497163,2024-04,paid,#891911,7.350000e+12,First-time,BB719-50,99.99,Wisconsin,2024-W14,2024-04,1,WI
497164,2024-04,paid,#892251,7.360000e+12,First-time,BB660B,84.99,Arizona,2024-W14,2024-04,1,AZ
497167,2024-04,paid,#892262,7.360000e+12,First-time,BB1151-29,69.99,Wisconsin,2024-W14,2024-04,1,WI
497193,2024-04,paid,#892276,7.180000e+12,First-time,SQ9178530,179.97,Nevada,2024-W14,2024-04,1,NV


In [131]:
unique_values = BB_filtered['units_per_transaction'].unique()

unique_values

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([1.])

In [119]:
# Find all rows in the DataFrame where any column has NaN values
nan_records = BB_filtered[BB_filtered.isna().any(axis=1)]
nan_records


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_id,customer_type,variant_sku,product_price,shipping_region,customer_cohort_week,customer_cohort_month,units_per_transaction,state_code
9713,2023-01,paid,#655741,6.420000e+12,First-time,SQ1726938,149.97,District Of Columbia,2023-W01,2023-01,1,NaN
11357,2023-01,paid,#660160,6.430000e+12,First-time,SQ5173256,99.98,District Of Columbia,2023-W02,2023-01,1,NaN
11907,2023-01,paid,#668354,6.450000e+12,First-time,SQ9178530,44.99,District Of Columbia,2023-W04,2023-01,1,NaN
23900,2023-01,paid,#670405,6.450000e+12,First-time,BB052-24,64.99,District Of Columbia,2023-W04,2023-01,1,NaN
28138,2023-01,paid,#669811,5.360000e+12,First-time,BB694-18,59.99,District Of Columbia,2023-W04,2023-01,2,NaN
29875,2023-01,paid,#664968,6.440000e+12,First-time,BB881-25,44.99,District Of Columbia,2023-W03,2023-01,1,NaN
30306,2023-01,paid,#655322,6.420000e+12,First-time,BB125-29,54.99,District Of Columbia,2023-W01,2023-01,1,NaN
30376,2023-01,paid,#660129,6.430000e+12,First-time,SQ9178530,134.97,District Of Columbia,2023-W02,2023-01,1,NaN
42589,2023-02,paid,#678696,6.470000e+12,First-time,BB903-29,69.99,District Of Columbia,2023-W06,2023-02,1,NaN
42622,2023-02,paid,#679790,6.480000e+12,First-time,BB052-24,69.99,District Of Columbia,2023-W06,2023-02,1,NaN


In [120]:
# Use the .loc method to target the 'state_code' column for rows where
# the 'shipping_region' is 'District Of Columbia'.
# This method allows us to select specific parts of the DataFrame based on a condition.
BB_filtered.loc[
   BB_filtered['shipping_region'] == 'District Of Columbia',  # Condition to filter rows
    'state_code'  # Column to apply the change
] = BB_filtered.loc[
    BB_filtered['shipping_region'] == 'District Of Columbia',  # Condition to filter rows again for consistency
    'state_code'  # Column to apply the .fillna() method
].fillna('DC')  # The .fillna() method replaces NaN values with 'DC'


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [121]:
# check for NaN values
nan_records = BB_filtered[BB_filtered.isna().any(axis=1)]
nan_records

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_id,customer_type,variant_sku,product_price,shipping_region,customer_cohort_week,customer_cohort_month,units_per_transaction,state_code


In [122]:
def remove_shipping_region(BB_filtered):
    # Remove the 'shipping_region' column
    BB_filtered.drop('shipping_region', axis=1, inplace=True)
    return BB_filtered

BB_filtered = remove_shipping_region(BB_filtered)

BB_filtered


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,month,financial_status,order_name,customer_id,customer_type,variant_sku,product_price,customer_cohort_week,customer_cohort_month,units_per_transaction,state_code
362,2023-01,paid,#658392,6.420000e+12,First-time,SQ8862610,54.99,2023-W01,2023-01,1,NY
377,2023-01,paid,#670349,6.450000e+12,First-time,BBNYBGLA,69.99,2023-W04,2023-01,1,VA
393,2023-01,paid,#666714,6.440000e+12,First-time,BB648B,109.99,2023-W03,2023-01,1,PA
430,2023-01,paid,#661459,6.430000e+12,First-time,BB062-25,49.99,2023-W02,2023-01,1,CT
459,2023-01,paid,#661431,6.430000e+12,First-time,BB736-37,64.99,2023-W02,2023-01,1,TX
...,...,...,...,...,...,...,...,...,...,...,...
497163,2024-04,paid,#891911,7.350000e+12,First-time,BB719-50,99.99,2024-W14,2024-04,1,WI
497164,2024-04,paid,#892251,7.360000e+12,First-time,BB660B,84.99,2024-W14,2024-04,1,AZ
497167,2024-04,paid,#892262,7.360000e+12,First-time,BB1151-29,69.99,2024-W14,2024-04,1,WI
497193,2024-04,paid,#892276,7.180000e+12,First-time,SQ9178530,179.97,2024-W14,2024-04,1,NV


In [124]:
BB_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23878 entries, 362 to 497196
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   month                  23878 non-null  object 
 1   financial_status       23878 non-null  object 
 2   order_name             23878 non-null  object 
 3   customer_id            23878 non-null  float64
 4   customer_type          23878 non-null  object 
 5   variant_sku            23878 non-null  object 
 6   product_price          23878 non-null  float64
 7   customer_cohort_week   23878 non-null  object 
 8   customer_cohort_month  23878 non-null  object 
 9   units_per_transaction  23878 non-null  int64  
 10  state_code             23878 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 2.7+ MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#Customer Lifetime Value (CLV)


In [125]:
#package used for customer lifetime value modeling
!pip install lifetimes
import lifetimes
import seaborn as sns
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_frequency_recency_matrix
# Set the maximum number of rows to display to 500.
pd.set_option('display.max_rows', 500)
# Set the maximum number of columns to display to 500.
pd.set_option('display.max_columns', 500)
# Set the width of the display in characters to 1000.
pd.set_option('display.width', 1000)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [126]:
BB_filtered.isnull().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


month                    0
financial_status         0
order_name               0
customer_id              0
customer_type            0
variant_sku              0
product_price            0
customer_cohort_week     0
customer_cohort_month    0
units_per_transaction    0
state_code               0
dtype: int64

In [128]:
BB_filtered.describe()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,customer_id,product_price,units_per_transaction
count,2.387800e+04,23878.000000,23878.000000
mean,6.868358e+12,130.041513,1.008669
std,3.743294e+11,154.917212,0.139283
min,1.520000e+12,1.990000,0.000000
25%,6.580000e+12,59.990000,1.000000
50%,6.970000e+12,69.990000,1.000000
75%,7.170000e+12,119.990000,1.000000
max,7.360000e+12,1184.880000,9.000000


In [103]:
BB_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23878 entries, 362 to 497196
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   month                  23878 non-null  object 
 1   financial_status       23878 non-null  object 
 2   order_name             23878 non-null  object 
 3   customer_type          23878 non-null  object 
 4   variant_sku            23878 non-null  object 
 5   product_price          23878 non-null  float64
 6   customer_cohort_week   23878 non-null  object 
 7   customer_cohort_month  23878 non-null  object 
 8   units_per_transaction  23878 non-null  int64  
 9   state_code             23878 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 2.5+ MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [104]:

def find_boundaries(df, variable,q1=0.05,q2=0.95):
    # the boundaries are the quantiles
    lower_boundary = df[variable].quantile(q1) # lower quantile
    upper_boundary = df[variable].quantile(q2) # upper quantile
    return upper_boundary, lower_boundary
def capping_outliers(df,variable):
    upper_boundary,lower_boundary =  find_boundaries(df,variable)
    df[variable] = np.where(df[variable] > upper_boundary, upper_boundary,
                       np.where(df[variable] < lower_boundary, lower_boundary, df[variable]))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Function to cap outliers in the 'Total_sales'
capping_outliers(BB_filtered, 'total_sales')

# Function to cap outliers in the 'units_per_transaction'
capping_outliers(BB_filtered, 'units_per_transaction')


In [129]:
# Function to cap outliers in the 'Total_sales'
capping_outliers(BB_filtered, 'product_price')

# Function to cap outliers in the 'units_per_transaction'
capping_outliers(BB_filtered, 'units_per_transaction')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [130]:
BB_filtered.describe()
BB_filtered.describe()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,customer_id,product_price,units_per_transaction
count,2.387800e+04,23878.000000,23878.0
mean,6.868358e+12,118.620368,1.0
std,3.743294e+11,112.376560,0.0
min,1.520000e+12,24.990000,1.0
25%,6.580000e+12,59.990000,1.0
50%,6.970000e+12,69.990000,1.0
75%,7.170000e+12,119.990000,1.0
max,7.360000e+12,444.940000,1.0


### Analysis Summary of Customer Transactions:

Our data set consists of transactional records which include customer IDs, product prices, and units per transaction. Here’s a quick summary of our findings:

- **Transactions Analyzed:** 23,878
- **Average Customer ID:** 6.86385e+12 (This number might represent a unique identifier system and does not convey any immediate customer insights)
- **Product Price Analysis:**
  - The **average price** of a product is approximately $116.60

 Prices exhibit a variation (standard deviation) of about $112.38 indicating

  a wide range of product prices.
  - The **minimum price** recorded is $24.99,
  
  showing our entry-level pricing point.
  - The **maximum price** observed is $444.94,
  
  indicating our premium product pricing point.
  - **25%** of our products are priced below $59.99,
  
  which could be considered as our most accessible price range.
  - The **median price** is $69.99,
  
  suggesting that half of our products are priced below this point, indicating a strong affordability factor.
  - **75%** of our products are priced below $119.99, highlighting the upper-middle range of our pricing strategy.

- **Units Per Transaction:** Consistently 1 unit per transaction, which could indicate individual purchases rather than bulk orders or that our sales strategy focuses on single-item sales.

### Implications for Business Strategy:

- **Pricing:** There's a broad spectrum of product prices, which suggests a diversified product range that can cater to various customer segments from budget to premium.
- **Product Offering:** The consistent single unit per transaction hints at potential to introduce bundled deals or promotions to encourage multiple purchases.
- **Market Positioning:** The data supports the positioning of our products across different market segments, from entry-level to high-end.


##Part 1: Running the Apriori Algorithm on the Complete Online Retail Dataset

### 1.Run the Apriori Algorithm: Apply the Apriori algorithm to the full dataset. Set parameters to identify item sets appearing in at least 1% of transactions.


### 2.Analysis of Specific Item Set: Filter to the item set containing "10 COLOUR SPACEBOY PEN." Analyze its support value.

### 3.Item Set Length Analysis: Add a column for item set length. Focus on sets of length two with support between 0.02 and 0.021. Compare these to previous results.

### 4.Support Value Distribution: Plot the distribution of support values for the identified item sets

##Part 2: Finding the Association Rules on the Complete Online Retail Dataset

### 1.Fit Association Rule Model: Use the full dataset to generate association rules. Apply the confidence metric with a minimum threshold of 0.6.

### 2.Count and Compare Association Rules: Count the number of derived rules. Compare this with previous exercises.

### 3.Plot Confidence vs. Support: Create a plot to visualize the relationship between confidence and support for the rules.

### 4.Analyze Rule Metrics: Examine the distributions of confidence, lift, leverage, and conviction in your rules.

##Report writing



*   Insights gained from the Apriori algorithm and association rule mining.
*   How these insights can influence business decisions like store layout and pricing.

*   A comparison between the two datasets.
*   Reflections on how data partitioning and increasing data volume impact results.



